<table align="left">
  <tr>
    <td><img src="hf_sandbox.png" alt="Sandbox" width="120"/></td>
    <td align="left"><h1>Lesson 3: Secure Code Execution</h1></td>
  </tr>
</table>


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code>  file:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Show behavior of custom python interpreter

In [1]:
#!pip install git+https://github.com/huggingface/smolagents.git

In [1]:
from smolagents.local_python_executor import LocalPythonExecutor

custom_executor = LocalPythonExecutor(["numpy"])

In [2]:
def run_capture_exception(command: str):
    try:
        custom_executor(harmful_command)
    except Exception as e:
        print("ERROR:\n", e)

In [3]:
# Example 1: non-defined command
# In Jupyter it works
!echo Bad command

Bad command


In [4]:
# In our interpreter, it does not.
harmful_command="!echo Bad command"
run_capture_exception(harmful_command)

ERROR:
 Code parsing failed on line 1 due to: SyntaxError
!echo Bad command
 ^
Error: invalid syntax (<unknown>, line 1)


In [5]:
[
    're',
    'queue',
    'random',
    'statistics',
    'unicodedata',
    'itertools',
    'math',
    'stat',
    'time',
    'datetime',
    'collections',
    'numpy'
]

['re',
 'queue',
 'random',
 'statistics',
 'unicodedata',
 'itertools',
 'math',
 'stat',
 'time',
 'datetime',
 'collections',
 'numpy']

In [6]:
# Example 2: os not imported
harmful_command="""
import os
exit_code = os.system("echo Bad command")
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'import os' due to: InterpreterError: Import of os is not allowed. Authorized imports are: ['collections', 're', 'statistics', 'queue', 'math', 'stat', 'numpy', 'itertools', 'time', 'random', 'unicodedata', 'datetime']


In [7]:
# Example 3: random._os.system not imported
harmful_command="""
import random
random._os.system('echo Bad command')
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'random._os.system('echo Bad command')' due to: InterpreterError: Forbidden access to module: os


In [8]:
# Example 4: infinite loop
harmful_command="""
while True:
    pass
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'while True:
    pass' due to: InterpreterError: Maximum number of 1000000 iterations in While loop exceeded


In [9]:
custom_executor = LocalPythonExecutor(["PIL"])

harmful_command="""
from PIL import Image

img = Image.new('RGB', (100, 100), color='blue')

i=0
while i < 10000:
    img.save('simple_image_{i}.png')
    i += 1
"""
# custom_executor(harmful_command)
# Let's not execute this but it would not error out, and it would bloat your system with images.

## Running in a sandbox

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

E2B_API_KEY = os.getenv("E2B_API_KEY")

In [6]:
openai_api_key = os.getenv("OPENAI_API_KEY")

from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, VisitWebpageTool, ToolCallingAgent, ToolCollection

model = LiteLLMModel(
    #model_id="openai/gpt-4",  
    model_id="openai/gpt-4o-mini",  
    max_tokens=2048
)

### CANNOT IMPORT THIS MODEL FROM smolagents

In [9]:
"""
from smolagents import CodeAgent, HfApiModel, Tool

model = HfApiModel()
"""
from smolagents import Tool

class VisitWebpageTool(Tool):
    name = "visit_webpage"
    description = (
        "Visits a webpage at the given url and reads its content as a markdown string. Use this to browse webpages."
    )
    inputs = {
        "url": {
            "type": "string",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "string"

    def __init__(self, max_output_length: int = 40000):
        super().__init__()
        self.max_output_length = max_output_length

    def forward(self, url: str) -> str:
        try:
            import re

            import requests
            from markdownify import markdownify
            from requests.exceptions import RequestException

            from smolagents.utils import truncate_content
        except ImportError as e:
            raise ImportError(
                "You must install packages `markdownify` and `requests` to run this tool: for instance run `pip install markdownify requests`."
            ) from e
        try:
            response = requests.get(url, timeout=20)
            response.raise_for_status()  # Raise an exception for bad status codes
            markdown_content = markdownify(response.text).strip()
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
            return truncate_content(markdown_content, self.max_output_length)

        except requests.exceptions.Timeout:
            return "The request timed out. Please try again later or check the URL."
        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

agent = CodeAgent(
    tools=[VisitWebpageTool()],
    model=model,
    executor_type="e2b",
    executor_kwargs={"api_key": E2B_API_KEY},
    max_steps=5
)

Initializing executor, hold on...

E2B is running

<div style="font-size: 14px; background-color: #fff9b0; padding: 12px; border-left: 4px solid #facc15;">
  <strong>Note:</strong> The agent is not always successful in this run.
</div>


In [10]:
output = agent.run(
    "Give me one of the top github repos from organization huggingface."
)
print("E2B executor result:", output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me one of the top github repos from organization huggingface.                                              │
│                                                                                                                 │
╰─ LiteLLMModel - openai/gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────╯

Collecting markdownify

  Downloading markdownify-1.1.0-py3-none-any.whl.metadata (9.1 kB)
Requirement already satisfied: requests in /usr/local/lib/python3.12/site-packages (2.32.4)
Requirement already satisfied: beautifulsoup4<5,>=4.9 in /usr/local/lib/python3.12/site-packages (from markdownify)
(4.13.4)
Requirement already satisfied: six<2,>=1.15 in /usr/local/lib/python3.12/site-packages (from markdownify) (1.17.0)
Requirement already satisfied: charset_normalizer<4,>=2 in /usr/local/lib/python3.12/site-packages (from requests) 
(3.4.2)
Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.12/site-packages (from requests) (3.10)
Requirement already satisfied: urllib3<3,>=1.21.1 in /usr/local/lib/python3.12/site-packages (from requests) 
(2.5.0)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.12/site-packages (from requests) 
(2025.7.9)
Requirement already satisfied: soupsieve>1.2 in /usr/local/lib/python3.12/site-packages (from 
beautifulsoup4<5,>=4.9->markdownify) (2.7)
Requirement already satisfied: typing-extensions>=4.0.0 in /usr/local/lib/python3.12/site-packages (from 
beautifulsoup4<5,>=4.9->markdownify) (4.14.1)

Downloading markdownify-1.1.0-py3-none-any.whl (13 kB)

Installing collected packages: markdownify
Successfully installed markdownify-1.1.0
[33mWARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the 
system package manager, possibly rendering your system unusable. It is recommended to use a virtual environment 
instead: https://pip.pypa.io/warnings/venv. Use the --root-user-action option if you know what you are doing and 
want to suppress this warning.[0m[33m
[0m

Executing code yielded an error:ModuleNotFoundError
No module named 
'smolagents'---------------------------------------------------------------------------ModuleNotFoundError         
Traceback (most recent call last)Cell In[3\], line 23
     21 import markdownify
     22 import requests
---> 23 import smolagents
     24 import re
     26 class VisitWebpageTool(Tool):
ModuleNotFoundError: No module named 'smolagents'

AgentError: Executing code yielded an error:ModuleNotFoundError
No module named 'smolagents'---------------------------------------------------------------------------ModuleNotFoundError                       Traceback (most recent call last)Cell In[3], line 23
     21 import markdownify
     22 import requests
---> 23 import smolagents
     24 import re
     26 class VisitWebpageTool(Tool):
ModuleNotFoundError: No module named 'smolagents'